In [ ]:
from transformers import LlamaForCausalLM, CodeLlamaTokenizer, BitsAndBytesConfig

In [30]:
import pandas as pd
import os
from transformers import AutoTokenizer

# You can initialize the tokenizer here. For example, for BERT:
tokenizer = CodeLlamaTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")

def get_files_in_dir(path, suffix='.txt'):
    """
    Prepares a dataset for next token prediction from files with a specific suffix in the given directory and its subdirectories.

    Parameters:
    - path (str): The directory path to look for files.
    - tokenizer: Tokenizer from the transformers library.
    - suffix (str): The file suffix to filter by. Default is '.txt'.
    - MAX_CONTENT_LENGTH (int): The maximum length for the input content.

    Returns:
    - pd.DataFrame: A DataFrame with 'input' and 'target' columns for next token prediction.
    """

    # Lists for storing input sequences and target tokens
    

    # Walking through the directory and its subdirectories
    rets = []
    for dirpath, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith(suffix):
                rets.append(os.path.join(dirpath, filename))
    return rets

def prepare_file_training(file_path: str, tokenizer: CodeLlamaTokenizer, MAX_CONTENT_LENGTH=512, MIN_CONTENT_LENGTH=128):
    inputs = []
    targets = []
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        content = file.read()
        # Tokenize the content using the provided tokenizer
        tokens = tokenizer.tokenize(content)
        # Start at min content length
        # for i in range(0, len(tokens) - MIN_CONTENT_LENGTH):  # -1 to leave space for the target token
            # print(i, len(tokens))
        for length in range(MIN_CONTENT_LENGTH, min(MAX_CONTENT_LENGTH, len(tokens)) + 1):
            i = 0
            # print(i, length, len(tokens))
            # print(i, length, len(tokens))
            input_sequence = tokens[i:min(i + length, len(tokens))]
            if i + length < len(tokens):
                target_token = tokens[i + length]
                inputs.append(tokenizer.convert_tokens_to_string(input_sequence))
                targets.append(target_token)
                
        # TODO: clean up
        for i in range(0, max(0, len(tokens) - MAX_CONTENT_LENGTH)):
            length = MAX_CONTENT_LENGTH
            input_sequence = tokens[i:min(i + length, len(tokens))]
            if i + length < len(tokens):
                target_token = tokens[i + length]
                inputs.append(tokenizer.convert_tokens_to_string(input_sequence))
                targets.append(target_token)

    return pd.DataFrame({'input': inputs, 'target': targets})


path = '../../../lean/mathlib4/Mathlib/'

MAX_CONTENT_LENGTH=2_048
lean_files = get_files_in_dir(path, ".lean")
df = prepare_file_training(lean_files[0], tokenizer, MAX_CONTENT_LENGTH, MIN_CONTENT_LENGTH=1)

In [34]:
n_files = len(lean_files)
n_train_files = 10

In [39]:
import numpy as np

np.random.seed(42)
test_file_paths = np.random.choice(np.arange(n_files), n_train_files).tolist()
def save_prepared_file(save_label: str, file_path: str, tokenizer: CodeLlamaTokenizer, MAX_CONTENT_LENGTH=512, MIN_CONTENT_LENGTH=128):
		df = prepare_file_training(file_path, tokenizer, MAX_CONTENT_LENGTH, MIN_CONTENT_LENGTH)
		save_path = os.path.join('data/train', save_label)
		df.to_json(save_path + '.json', index=False)

for file_idx in test_file_paths:
	file_path = lean_files[file_idx]
	save_prepared_file('testing_' + str(file_idx), file_path, tokenizer, MAX_CONTENT_LENGTH, MIN_CONTENT_LENGTH=1)


In [26]:
print(df.target[7]), print(df.input[8])

actic
import Mathlib.Tactic.Abel
import Mathlib.Tactic.ApplyCongr
import Mathlib.Tactic.ApplyFun
import Mathlib.Tactic.ApplyWith
import Mathlib.Tactic.Attr.Core
import Mathlib.Tactic.Attr.Register
import Mathlib.Tactic.Backtrack
import Mathlib.Tactic.Basic
import Mathlib.Tactic.ByContra
import Mathlib.Tactic.Cache
import Mathlib.Tactic.CancelDenoms
import Mathlib.Tactic.CancelDenoms.Core
import Mathlib.Tactic


(None, None)